In [5]:
import numpy as np
from bert4keras.backend import keras, set_gelu
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam, extend_with_piecewise_linear_lr
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open
from keras.layers import Lambda, Dense
import pandas as pd
import random
set_gelu('tanh')  # 切换gelu版本


In [6]:
num_classes = 2
maxlen = 128
batch_size = 32
config_path = '../model/chinese_wwm_ext_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '../model/chinese_wwm_ext_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '../model/chinese_wwm_ext_L-12_H-768_A-12/vocab.txt'

In [7]:
def load_data(valid_rate=0.3):
    train_file = "../data/train.csv"
    test_file = "../data/test.csv"
    
    df_train_data = pd.read_csv("../data/train.csv")
    df_test_data = pd.read_csv("../data/test.csv")
    
    train_data, valid_data, test_data = [], [], []
    
    for row_i, data in df_train_data.iterrows():
        id, level_1, level_2, level_3, level_4, content, label = data
        
        id, text, label = id, str(level_1) + '\t' + str(level_2) + '\t' + \
        str(level_3) + '\t' + str(level_4) + '\t' + str(content), label
        
        if random.random() > valid_rate:
            train_data.append( (id, text, int(label)) )
        else:
            valid_data.append( (id, text, int(label)) )
            
    for row_i, data in df_test_data.iterrows():
        id, level_1, level_2, level_3, level_4, content = data
        
        id, text, label = id, str(level_1) + '\t' + str(level_2) + '\t' + \
        str(level_3) + '\t' + str(level_4) + '\t' + str(content), 0
        
        test_data.append( (id, text, int(label)) )
    return train_data, valid_data, test_data

In [8]:
train_data, valid_data, test_data = load_data(valid_rate=0.3)

In [9]:
# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

In [10]:
# class data_generator(DataGenerator):
#     """数据生成器
#     """
#     def __iter__(self, random=False):
#         batch_token_ids, batch_segment_ids, batch_labels = [], [], []
#         for is_end, (text, label) in self.sample(random):
#             token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
#             batch_token_ids.append(token_ids)
#             batch_segment_ids.append(segment_ids)
#             batch_labels.append(label)
#             if len(batch_token_ids) == self.batch_size or is_end:
#                 batch_token_ids = sequence_padding(batch_token_ids)
#                 batch_segment_ids = sequence_padding(batch_segment_ids)
#                 batch_labels = to_categorical(batch_labels, num_classes)
#                 yield [batch_token_ids, batch_segment_ids], batch_labels
#                 batch_token_ids, batch_segment_ids, batch_labels = [], [], []
class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, (id, text, label) in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append([label])
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []

# 转换数据集
train_generator = data_generator(train_data, batch_size)
valid_generator = data_generator(valid_data, batch_size)
test_generator = data_generator(test_data, batch_size)

In [12]:
# 加载预训练模型
bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    model='roberta',
    return_keras_model=False,
)
output = Lambda(lambda x: x[:, 0], name='CLS-token')(bert.model.output)
output = Dense(
    units=num_classes,
    activation='softmax',
    kernel_initializer=bert.initializer
)(output)

model = keras.models.Model(bert.model.input, output)
model.summary()


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 768)    16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [13]:
# 派生为带分段线性学习率的优化器。
# 其中name参数可选，但最好填入，以区分不同的派生优化器。
AdamLR = extend_with_piecewise_linear_lr(Adam, name='AdamLR')

model.compile(
    loss='sparse_categorical_crossentropy',
    # optimizer=Adam(1e-5),  # 用足够小的学习率
    optimizer=AdamLR(learning_rate=1e-4, lr_schedule={
        1000: 1,
        2000: 0.1
    }),
    metrics=['accuracy'],
)


In [14]:

# def evaluate(data):
#     total, right = 0., 0.
#     for x_true, y_true in data:
#         y_pred = model.predict(x_true).argmax(axis=1)
#         y_true = y_true.argmax(axis=1)
#         total += len(y_true)
#         right += (y_true == y_pred).sum()
#     return right / total

def evaluate(data):
    total, right = 0., 0.
    for x_true, y_true in data:
        y_pred = model.predict(x_true).argmax(axis=1)
        y_true = y_true[:, 0]
        total += len(y_true)
        right += (y_true == y_pred).sum()
    return right / total



class Evaluator(keras.callbacks.Callback):
    """评估与保存
    """
    def __init__(self):
        self.best_val_acc = 0.

    def on_epoch_end(self, epoch, logs=None):
        val_acc = evaluate(valid_generator)
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            model.save_weights('best_model.weights')
#         test_acc = evaluate(test_generator)
#         print(
#             u'val_acc: %.5f, best_val_acc: %.5f, test_acc: %.5f\n' %
#             (val_acc, self.best_val_acc, test_acc)
#         )
        test_acc = evaluate(valid_generator)
        print(
            u'val_acc: %.5f, best_val_acc: %.5f, test_acc: %.5f\n' %
            (val_acc, self.best_val_acc, test_acc)
        )

263

In [16]:
evaluator = Evaluator()

In [17]:
model.fit(
        train_generator.forfit(),
        steps_per_epoch=len(train_generator),
        epochs=10,
        callbacks=[evaluator]
    )

/root/anaconda3/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


Epoch 1/10
263/263 [==============================] - 138s 523ms/step - loss: 0.1912 - accuracy: 0.9139
val_acc: 0.97798, best_val_acc: 0.97798, test_acc: 0.97798

Epoch 2/10
263/263 [==============================] - 127s 482ms/step - loss: 0.0754 - accuracy: 0.9743
val_acc: 0.97547, best_val_acc: 0.97798, test_acc: 0.97547

Epoch 3/10
263/263 [==============================] - 126s 480ms/step - loss: 0.0585 - accuracy: 0.9825
val_acc: 0.97770, best_val_acc: 0.97798, test_acc: 0.97770

Epoch 4/10
263/263 [==============================] - 127s 482ms/step - loss: 0.0805 - accuracy: 0.9735
val_acc: 0.92224, best_val_acc: 0.97798, test_acc: 0.92224

Epoch 5/10
263/263 [==============================] - 127s 481ms/step - loss: 0.0915 - accuracy: 0.9697
val_acc: 0.97742, best_val_acc: 0.97798, test_acc: 0.97742

Epoch 6/10
263/263 [==============================] - 126s 480ms/step - loss: 0.0402 - accuracy: 0.9879
val_acc: 0.97770, best_val_acc: 0.97798, test_acc: 0.97770

Epoch 7/10
263/2

In [24]:
model.load_weights('best_model.weights')
# print(u'final test acc: %05f\n' % (evaluate(test_generator)))
print(u'final test acc: %05f\n' % (evaluate(valid_generator)))

final test acc: 0.979933



In [25]:
print(u'final test acc: %05f\n' % (evaluate(train_generator)))

final test acc: 0.997741



In [26]:
def data_pred3(test_data):
    id_ids, y_pred_ids = [], []
    for id, text, label in test_data:
        token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
        token_ids = sequence_padding([token_ids])
        segment_ids = sequence_padding([segment_ids])
        y_pred = int(model.predict([token_ids, segment_ids]).argmax(axis=1)[0])
        id_ids.append(id)
        y_pred_ids.append(y_pred)
    return id_ids, y_pred_ids

In [27]:
id_ids, y_pred_ids = data_pred3(test_data)

In [28]:
df_save = pd.DataFrame()
df_save['id'] = id_ids
df_save['label'] = y_pred_ids

In [23]:
df_save.to_csv('result.csv')